In [95]:
import os 
from glob import glob 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn 
from torch.utils.data import Dataset,DataLoader
import torchvision
from torchvision import transforms as T 
import cv2 
import torch.nn.functional as F 

In [2]:
dir_default = './data/open/'


def file_load(dir_default):
    dir_train_imgs = sorted(glob(os.path.join(dir_default,'train','*')))
    dir_test_imgs = sorted(glob(os.path.join(dir_default,'test','*')))
    train_df = pd.read_csv(os.path.join(dir_default,'train_df.csv'))
    test_df = pd.read_csv(os.path.join(dir_default,'test_df.csv'))
    return dir_train_imgs,dir_test_imgs,train_df, test_df 

In [53]:
dir_train_imgs,dir_test_imgs,train_df, test_df  = file_load(dir_default)

#라벨 생성 
train_labels = train_df['label'].to_numpy()
#라벨 인코더 
label_encoder = {b:a for a,b in enumerate(np.unique(train_labels))}
label_decoder = {b:a for a,b in enumerate(np.unique(train_labels))}

(4277,)

# 데이터셋 

In [72]:
class Dset(Dataset):
    def __init__(self,dir_imgs,labels):
        self.dir_imgs = dir_imgs 
        self.labels = labels 
        self.transform = self.transform_img()

    def __len__(self):
        return len(self.dir_imgs)

    def transform_img(self,train=True):
        if train:
            transform = T.Compose([
                T.ToTensor(),
                T.Resize(256),
                T.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                    )
            ])
        return transform 

    def load_img(self,dir_img):
        temp_img = cv2.imread(dir_img)
        temp_img = self.transform(temp_img)
        return temp_img 

    def __getitem__(self,idx):
        img = self.load_img(self.dir_imgs[idx])
        label = label_encoder[self.labels[idx]]
        return img, label 

In [73]:
train_dset = Dset(dir_train_imgs,train_labels)
train_loader = DataLoader(train_dset,batch_size=16,shuffle=True)
a,b = next(iter(train_loader))


# 모델 구성 

In [100]:
class clf(nn.Module):
    def __init__(self):
        super(clf, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(256,128),
            nn.Linear(128,64),
            nn.Linear(64,2),
            nn.Flatten(),
            nn.Linear(1536,88)
        )
    def forward(self,x):
        
        x = self.fc1(x)
        x = F.softmax(x,dim=0)
        return x 

In [108]:
torch.argmax(model(a)[4])

tensor(83)